<a href="https://colab.research.google.com/github/srulikbd/Deep-Learning/blob/master/Paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 
!pip install torch==1.7
!pip install transformers==3.5
!pip install pytorch_lightning==0.7.5

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

sentence = "“Georgia is not the only state that pushed through these last minute rules changes before the presidential election.” @marthamaccallum They forgot about our Constitution!"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=10
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

device  cuda


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."



Original Question ::
“Georgia is not the only state that pushed through these last minute rules changes before the presidential election.” @marthamaccallum They forgot about our Constitution!


Paraphrased Questions :: 
0: How did Georgia get on my list with the last minute rules change before the election?
1: Because Georgia is a state, it is very unusual to see states putting in last minute changes so before the 2016 election. I'm sorry, Virginia, who hasn't complied with the first amendments?
2: I know Georgia was upset by last minute change in rules that President Obama allowed before the election. I'd feel sorry for Georgia’s attitude. Are there any other states that have done this before?
3: What are some states that forgot about our Constitution and their rules before the election?
4: Is Georgia the only state in the world that pushed through these last minute rules changes for the 2016 presidential election?
5: "Georgia is not the only state that pushed through these last minu

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)
sentence = "Georgia is not the only state that pushed through these last minute rules changes before the presidential election"

text =  "paraphrase: " + sentence + " </s>"

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")


outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

device  cuda


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Georgia is not the only state that pushed through these last minute rules changes before the presidential election.
Georgia is not the only state that pushed these last minute rules changes before the presidential election to the public.
Georgia is not the only state that pushed these last-minute rule changes before the presidential election.
Georgia is not the only state that has pushed through these last minute rule changes before the presidential election
Georgia is not the only state that pushed these last minute changes to the rules before the presidential election.


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=120,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.0)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

context = "Still the greatest President in the history of our once Great country. MAGA NOT GIVING UP! He was born to run, don't belong to anyone. He don't need to be loved by you. Fire in His lungs, can't bite the devil on His tongue. Because He is a big macho dragon man."
get_response(context,10)

['He is the greatest President in the history of our country.',
 "He doesn't need to be loved by you, he was born to run.",
 'He is the greatest President in the history of the country.',
 'He is the greatest President in the history of the United States.',
 'He is still the greatest President in the history of our country.',
 "He doesn't need to be loved by you, he was born to run, and can't bite the devil.",
 "He's the greatest President in the history of our country.",
 "He doesn't need to be loved by you, he was born to run, and he can't bite the devil.",
 "He's the greatest President in the history of the country.",
 "He's still the greatest President in the history of our country."]